## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [2]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [3]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.003813645918853581
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.15939615399111062
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.008109500049613416
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.23655258293729275


## Different MatMul Operations

In [2]:
from max.driver import CPU, Accelerator, accelerator_count, Tensor
import torch
M = 4096
K = 6144
N = 2048
device = CPU() if accelerator_count() == 0 else Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [3]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
print('building cuda graph...')
graph =  build_graph(session, "naive")
mojo_result = graph.execute(A, B)[0].to_numpy() # test run

building cuda graph...
algo: naive


Now verify the kernel calculation:

In [4]:
print(mojo_result, end="\n\n")
print("reference:\n", torch_result)
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.002)

[[ -10.648993     48.60996      -0.19582635 ...  150.46631
    70.85625      -3.4492505 ]
 [ -28.453247     15.283995    -32.81223    ...   28.671976
    46.407955    109.3716    ]
 [ 139.75092       4.9468503   -50.545868   ...   68.71995
    66.24626     -52.164837  ]
 ...
 [ -40.99045     -93.46522     -39.555977   ...  -14.283063
    67.841934     28.53191   ]
 [ 176.59221    -133.46938      94.15844    ...   83.28419
   -54.039482   -121.6142    ]
 [ 119.07402     -27.128508    -79.48105    ...   12.437745
    46.40625      61.38567   ]]

reference:
 [[ -10.648945     48.61         -0.19587421 ...  150.46626
    70.85631      -3.449338  ]
 [ -28.453346     15.284033    -32.812153   ...   28.672068
    46.40795     109.3716    ]
 [ 139.75099       4.946871    -50.545883   ...   68.72
    66.24629     -52.16492   ]
 ...
 [ -40.99041     -93.46512     -39.555916   ...  -14.282967
    67.84189      28.531898  ]
 [ 176.59218    -133.46962      94.15844    ...   83.284225
   -54.039413 

Test the speed?

In [9]:
import time
for algo in ["naive", "coalescing"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        begin = time.perf_counter()
        torch_result = (torch_A @ torch_B).cpu().numpy()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)[0].to_numpy()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.002)
    print(algo, record)

naive {'torch': 0.018712376605253666, 'mojo': 0.5130531819886528}
coalescing {'torch': 0.01883281220216304, 'mojo': 0.13362077759811655}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul